In [1]:
from fastai.vision.all import *

In [2]:
learner = load_learner('./learns/fruits.pkl')

In [3]:
dls = ImageDataLoaders.from_folder(
    "./dataset/train",
    valid_pct=0.2,
    seed=42,
    item_tfms=Resize(224)
)

In [4]:
image = PILImage.create("./dataset/test/Strawberry/Strawberry (1423).jpeg")
image.show()

In [5]:
learner.predict(image)

# Grad-CAM

In [6]:
x, = first(dls.test_dl([image]))

In [7]:
class Hook:
    def hook_func(self, m, i, o):
        self.stored = o.detach().clone()

In [8]:
hook_output = Hook()
hook = learner.model[0].register_forward_hook(hook_output.hook_func)

In [9]:
with torch.no_grad(): output = learner.model.eval()(x)

In [10]:
act = hook_output.stored[0]
F.softmax(output, dim=-1)

In [11]:
dls.vocab

In [12]:
act.shape

In [13]:
cam_map = torch.einsum('ck,kij->cij', learner.model[1][-1].weight, act)
cam_map.shape

In [14]:
x_dec = TensorImage(dls.train.decode((x,))[0][0])
_ , ax = plt.subplots()
x_dec.show(ctx=ax)
ax.imshow(cam_map[1].detach().cpu(), alpha=0.6, extent=(0,224,224,0),
              interpolation='bilinear', cmap='magma');

In [15]:
hook.remove()

In [16]:
class Hook():
    def __init__(self, m):
        self.hook = m.register_forward_hook(self.hook_func)
    def hook_func(self, m, i, o): self.stored = o.detach().clone()
    def __enter__(self, * args): return self
    def __exit__(self, *args): self.hook.remove()

# Grad-CAM

In [17]:
class HookBwd:
    def __init__(self, m):
        self.hook = m.register_backward_hook(self.hook_func)
    def hook_func(self, m, gi, go): self.stored = go[0].detach().clone()
    def __enter__(self, *args): return self
    def __exit__(self, *args): return self.hook.remove()

In [18]:
cls = 1
with HookBwd(learner.model[0]) as hookg:
    with Hook(learner.model[0]) as hook:
        output = learner.model.eval()(x.cuda())
        act = hook.stored
    output[0, cls]. backward()
    grad = hookg.stored

In [ ]:
w = grad[0].mean(dim=[1,2], keepdim=True)
cam_map = (w * act[0]).sum(0)

_, ax = plt.subplots()
x_dec.show(ctx=ax)
ax.imshow(cam_map.detach().cpu(), alpha=0.6, extent=(0,224,224,0),
          interpolation='bilinear', cmap='jet');